https://doc.sagemath.org/html/en/reference/rings_numerical/sage/rings/real_mpfi.html

https://doc.sagemath.org/html/en/reference/spkg/mpfi.html#spkg-mpfi

https://doc.sagemath.org/html/en/reference/spkg/mpfr.html#spkg-mpfr

In [8]:
prec = 200
RBF = RealBallField(prec)
RIF = RealIntervalField(prec)

I = RIF(-1,1)
J=RBF(-1,1)


def g(x):
    return exp(x)


print(abs(g(I).upper()))
print(abs(g(J).upper()))

2.7182818284590452353602874713526624977572470936999595749670
1.0000000065192580223083496093750000000000000000000000000000


In [ ]:
f.taylor(x,2,1)

x |--> 1

In [ ]:
phi_1_d1_0=diff(phi_list[1],x).taylor(x,0)
phi_1_d1_func=smooth_fast_callable(diff(phi_list[1],x),x,phi_1_d1_0)

phi_1_d2_0=diff(diff(phi_list[2],x),x).taylor(x,0)
phi_1_d1_func=smooth_fast_callable(diff(diff(phi_list[2],x),x),x,phi_1_d2_0)


def M_0_func(ball1,ball2):
    M_fast = fast_callable(M_0, vars=[x,y], domain=RBF)
    if ball1==ball2 :
        return (lambda_list_0[0] * phi_funcs[0](ball2))
    if ball2==RBF(0):
        return (lambda_funcs[0](ball1-ball2) * phi_list_0[0])
    else :
        return M_fast(ball1,ball2)
    
M_1 = (lambda_list[1](x=x - y) * diff(phi_list[1], x)(x=y) + lambda_list[2](x=x - y) * phi_list[1](x=y))

def M_1_func(ball1, ball2):
    M_fast = fast_callable(M_1, vars=[x,y], domain=RBF)
    if ball1 == ball2:
        return lambda_list_0[1] * diff(phi_list[0], x)(x=ball2) + lambda_list_0[2] * phi_list_0[1]
    if ball2 == RBF(0):
        # cas ball2 == 0
        return lambda_list[1](ball1 - ball2) * diff(phi_list[0], x)(x=ball2) + lambda_list[2](ball1 - ball2) * phi_list_0[1]
    else:
        return M_fast(ball1, ball2)


def M_2_func(ball1, ball2):
    M_fast = fast_callable(M_2, vars=[x,y], domain=RBF)
    if ball1 == ball2:
        # cas ball1 == ball2, valeurs en 0
        return lambda_list_0[3] * diff(diff(phi_list[2], x), x)(x=ball2) + lambda_list_0[3] * diff(phi_list[2], x)(x=ball2) + lambda_list_0[4] * phi_list_0[2]
    if ball2 == RBF(0):
        # cas ball2 == 0
        return (lambda_list[3](ball1 - ball2) * diff(diff(phi_list[2], x), x)(x=ball2) +
                lambda_list[3](ball1 - ball2) * diff(phi_list[2], x)(x=ball2) +
                lambda_list[4](ball1 - ball2) * phi_list_0[2])
    else:
        return M_fast(ball1, ball2)




#M_funcs=[smooth_convert_2D_safe(expr,x,y) for expr in M_list]